In [1]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key-HW",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images-HW" , static_folder = "./images-HW")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'
@app.route('/cxcxc')
def hello():
    return "基隆chatbot大賽等你哦"

In [2]:
'''
當用戶輸入文字的時候，回應用戶文字消息
'''
from linebot.models import MessageEvent,TextMessage,TextSendMessage
@handler.add(MessageEvent, message = TextMessage)
def handle_text_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage("謝謝")
    )

In [3]:
'''
當用戶傳入圖片的時候，下載圖片，以OPENCV改圖，傳回用戶，存在本地
'''
import cv2
import numpy as np
from linebot.models import MessageEvent,TextMessage,TextSendMessage,ImageMessage,ImageSendMessage
@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):        
    message_content = line_bot_api.get_message_content(event.message.id)
    path = './images-HW/'+event.message.id+'.jpg'
    with open(path, 'wb') as fd:        
        for chunk in message_content.iter_content():
            fd.write(chunk)
    intense = cv2.imread('./images-HW/Intense_Effect.jpg',-1)
    img = cv2.imread(path,-1)
    resized_img = cv2.resize(img,(1280,720))
    intensified = cv2.add(resized_img,intense)
    cv2.imwrite('./images-HW/Intensified_'+event.message.id+'.jpg',intensified,[cv2.IMWRITE_JPEG_QUALITY, 100])
    line_bot_api.reply_message(
        event.reply_token,
        [
            TextSendMessage(text='收到圖片啦，我要改圖囉。\n'+ '圖片ID:' + event.message.id),
            ImageSendMessage(
                original_content_url='https://%s/images-HW/Intensified_%s.jpg' % (server_url,event.message.id),
                preview_image_url='https://%s/images-HW/Intensified_%s.jpg' % (server_url,event.message.id)
            )
        ]
    )

In [4]:
event = '10502996129263'
original_content_url='https://%s/images-HW/Intensified_%s.jpg' % (server_url,event)
original_content_url

'https://6abcad0d.ngrok.io/images-HW/Intensified_10502996129263.jpg'

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Sep/2019 11:36:27] "GET //images-HW/Intensified_10503151710605.jpg HTTP/1.1" 404 -
127.0.0.1 - - [03/Sep/2019 11:36:29] "GET //images-HW/Intensified_10503151710605.jpg HTTP/1.1" 404 -
127.0.0.1 - - [03/Sep/2019 11:36:29] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2019 11:36:30] "GET /images-HW/Intensified_10503261029460.jpg HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2019 11:36:31] "GET /images-HW/Intensified_10503261029460.jpg HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2019 11:36:31] "GET /images-HW/Intensified_10503261029460.jpg HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2019 11:36:37] "GET /images-HW/Intensified_10503261029460.jpg HTTP/1.1" 200 -
